## First Weld Model of Detection

Til the date of this push in GIT, **the suggestion is run de code in Colab GPU T4** to reason of computacional cost.

* Download a dataset, prepare it, train it, and save it.

### 1) Load Dataset 🤓

We will use a set of images of Industrial Welding Defects provided by the client.

In [ ]:
!unrar x "Weld_Defects.rar"

### 2) Dependencies 👌

In [ ]:
!pip install ultralytics

In [ ]:
from ultralytics import YOLO

### 3) Training 💪

Load a model pre-trained

In [ ]:
model = YOLO("yolo11n.pt")

In [ ]:
yaml_path = "/content/defectos_de_soldaduras/data.yaml"

#### 📍 Option  1: Default Setup

In [ ]:
model.train(
    data=yaml_path,
    epochs=10,
    imgsz=640, # We need de Iamages dimensions
    augment=True,
    )

#### 📍 Option 2) Patience Early Stopping

If it doen't improve with 5 epochs in the loss validation, You must to stop training.

In [ ]:
model.train(
    data=yaml_path,
    epochs=20,
    imgsz=640,
    augment=True,
    patience=5,
    )

#### 📍 Option 3) Custom Data Augmentation

Diferentes operaciones de Data Augmentation.

In [ ]:
model.train(
    data=yaml_path,
    epochs=10,
    imgsz=640,
    augment=True,
    patience=5,
    hsv_h=0.015,    # HSV hue adjustment (change color tone)
    hsv_s=0.7,      # HSV saturation adjustment
    hsv_v=0.4,      # HSV brightness adjustment
    flipud=0.5,     # Vertical flip with 50% probability
    fliplr=0.5,     # Horizontal flip with 50% probability
    mosaic=1.0,     # Enable mosaic (image mixing)
    mixup=0.5,      # Mix two images with 50% probability
    perspective=0.0005,  # Perspective transformation
    degrees=5.0,    # Rotation of ±5 degrees
    shear=2.0,      # Image shearing
    scale=0.5,      # Image zoom
)

### 4) Save Model 💾

Save complet model ( Arquitecture + Weights )

In [ ]:
model.save("mi_modelo.pt")

Luego, para cargarlo:

In [ ]:
mi_modelo = YOLO("mi_modelo.pt")

### 5) Make predictions on production 🤙

#### 📍 1) View results

In [ ]:
import os
from IPython.display import Image, display

Create folder to save test results 

In [ ]:
save_dir = "./testing/"
os.makedirs(save_dir, exist_ok=True)

In [ ]:
image_path = "/content/defectos_de_soldaduras/valid/images/SampleV1_1_mp4-24_jpg.rf.8487d87bb4c1d1ab9059da84ac881495.jpg"

results = mi_modelo(image_path)

for result in results:
  result.plot(save=True, filename=os.path.join(save_dir, os.path.basename(result.path)))
  result_image_path = os.path.join(save_dir, os.path.basename(result.path))
  display(Image(filename=result_image_path))

#### 📍 2) Filter response

Return only wich ones with confidence up to 30%.

In [ ]:
image_path = "/content/defectos_de_soldaduras/valid/images/SampleV1_1_mp4-24_jpg.rf.8487d87bb4c1d1ab9059da84ac881495.jpg"

results = mi_modelo(
    image_path,
    conf=0.30,
    )

for result in results:
  result.plot(save=True, filename=os.path.join(save_dir, os.path.basename(result.path)))
  result_image_path = os.path.join(save_dir, os.path.basename(result.path))
  display(Image(filename=result_image_path))

Return only those with a confidence level greater than 30% and of the "Good Weld" class.

In [ ]:
image_path = "/content/defectos_de_soldaduras/valid/images/SampleV1_1_mp4-24_jpg.rf.8487d87bb4c1d1ab9059da84ac881495.jpg"


results = mi_modelo(
    image_path,
    conf=0.30,
    classes=[1],
    )

for result in results:
  result.plot(save=True, filename=os.path.join(save_dir, os.path.basename(result.path)))
  result_image_path = os.path.join(save_dir, os.path.basename(result.path))
  display(Image(filename=result_image_path))

#### 📍 3) Return Specific Results

In [ ]:
def detectar_objetos(image_path, model):
    """
    Performs object detection on an image and returns:
      - The bounding boxes in [x1, y1, x2, y2] format.
      - The names of the detected classes.
      - The confidences of each detection.
      - The annotated image with the detections drawn.

    Parameters:
      image_path (str): Path to the image.
      model: Trained YOLO model.

    Returns:
      boxes (numpy.ndarray): Array of bounding boxes.
      class_names (list): List with the names of the detected classes.
      confidences (numpy.ndarray): Array with detection scores/confidences.
      annotated_img (numpy.ndarray): Image with the detections drawn.
    """
    # Perform detection (for a single image, results is a list, we take the first result)
    results = model(image_path)
    result = results[0]

    # Extract bounding boxes in xyxy format
    boxes = result.boxes.xyxy.cpu().numpy()

    # Extract class indices and confidence scores
    classes_idx = result.boxes.cls.cpu().numpy().astype(int)
    confidences = result.boxes.conf.cpu().numpy()

    # If the model has a class name dictionary, retrieve class names
    if hasattr(model, "names"):
        class_names = [model.names[i] for i in classes_idx]
    else:
        class_names = classes_idx  # If not available, return class indices

    # Get the image with bounding boxes drawn on it
    annotated_img = result.plot()

    return boxes, class_names, confidences, annotated_img

In [ ]:
image_path = "/content/defectos_de_soldaduras/valid/images/SampleV2_1_mp4-26_jpg.rf.a8431cb1acce1c55d32861c982c2a16a.jpg"

boxes, classes, confs, img_anotada = detectar_objetos(image_path, mi_modelo)

In [ ]:
img_anotada

In [ ]:
print("Bounding Boxes:", boxes)
print("Class Names:", classes)
print("Confidences:", confs)

### 6) Conclusions

- learn about the different objects and methods that YOLO offers us.

- Perform the complete image detection process with YOLO.

<br>
<br>
<br>